#

[]{.linea-superior} 
[]{.linea-inferior} 

<img src="imagenes/logo_portada2.png" width="20%"/>  


[**Ciencia de datos para el sector público:**]{.big-par .center-justified}
[**Aplicación "Dónde está el censo" y su stack tecnológico**]{.big-par .center-justified}
[**Proyecto Ciencia de Datos**]{.medium-par.center-justified}

[**Abril 2024**]{.big-par .center-justified}



## Contenidos

- Contexto y objetivo
- Demostración de la aplicación
- Desafíos
- Estructura de la aplicación
- Base de Datos: PostGIS
- Flask + Frontend
- Gunicorn
- Nginx
- Orquestado de contenedores
- Conclusiones



## Contexto y objetivo

::: incremental

![](imagenes/logo-censo.png){.float-right width=25%  }

- En Chile, entre marzo y junio de 2024 se está ejecutando el Censo de Población y Vivienda.

- Por primera vez es un censo de derecho, durando 3 meses, en vez de un día.

- Esto implica que debió crearse una planificación sobre qué zonas serían censadas en distintos períodos.

- En la práctica, cada comuna (municipio) se divide en 4 ciclos que van siendo encuestados secuencialmente, 
en la medida que avanza el censo.

- En este contexto, se nos solicitó desarrollar una aplicación que mostrara qué sectores están siendo 
censados para la comuna seleccionada por el usuario.

:::


## Demostración de la aplicación


[Veamos cómo funciona la aplicación!](https://dondeestaelcenso.ine.gob.cl)


## Desafíos {.medium-par}

::: incremental

- Principalmente: es una aplicación de uso masivo, lo que implica que esta tiene que:
  - Ser capaz de escalar.
  - Ser eficiente.
  - Tener altas medidas de seguridad.
- Además, esta debe poder adaptarse: por su naturaleza, el censo está sujeto a contingencias
que pueden implicar cambios en la aplicación.
- Iremos explorando cómo los solucionamos durante la presentación

:::  


## Estructura de la aplicación

![](imagenes/diagrama.PNG)



## Base de Datos: PostGIS {.medium-par}

::: incremental
- Extiende las capacidades de una base de datos relacional PostgreSQL, agregando soporte para almacenamiento, indexado y consulta a datos geospaciales.

- Principales características:
  - Almacenamiento: puntos, líneas, polígonos y multi-geometrías, en 2D y 3D.
  - Indexado: permite rápida consulta y respuesta.
  - Funciones y Procesamiento geométrico: medición de distancias y áreas, intersecciones, buffers, simplificación, geocoding, etc.

- Para optimizar las consultas a la BBDD, creamos tres tablas que contienen diferentes datos geospaciales preprocesados con propósitos definidos en función de la operación de la aplicación.

- Esto permite que la BBDD sea estática: al ser consultada por la aplicación sólo retorna lo necesario, pero no realiza procesamientos demandantes.

:::



## Base de Datos: PostGIS {.medium-par}

::: incremental
- La BBDD se puebla a partir de scripts **.sql** creados con la herramienta **shp2pgsql**.
  - [https://postgis.net/docs/using_postgis_dbmanagement.html](https://postgis.net/docs/using_postgis_dbmanagement.html)

:::



## Base de Datos: PostGIS {.medium-par}


- La BBDD se puebla a partir de scripts **.sql** creados con la herramienta **shp2pgsql**.
  - [https://postgis.net/docs/using_postgis_dbmanagement.html](https://postgis.net/docs/using_postgis_dbmanagement.html)

![](imagenes/postgis_shp2pgsql.png){width="120%"}

::: {.small-par-2}


In [ ]:
#| echo: TRUE
#| eval: FALSE

# convert geodataframe to shape
gdf_db.to_file(path_shp)

# create script with sql inserts that populates table perimeters_arc
cmd = f"shp2pgsql -s 4674 -a -g geometry -W LATIN1 -I {path_shp} public.perimeters_arc > ../../postgis/sql/init/30_2_append_perimeters_arc.sql"
res = subprocess.call(cmd, shell=True)

# create script with sql inserts that populates table perimeters_arc_ciclo
cmd = f"shp2pgsql -s 4674 -a -g geometry -W LATIN1 -I {path_shp} public.perimeters_arc_ciclo > ../../postgis/sql/init/31_2_append_perimeters_arc_ciclo.sql"
res = subprocess.call(cmd, shell=True)

# create script with sql inserts that populates table comunas
cmd = f"shp2pgsql -s 4674 -a -g geometry -W LATIN1 -I {path_shp} public.comunas > 40_2_append_comunas.sql"
res = subprocess.call(cmd, shell=True)

:::



## Base de Datos: PostGIS {.medium-par}

- La BBDD se puebla a partir de scripts **.sql** creados con la herramienta **shp2pgsql**.
  - Ejemplo de script **40_2_append_comunas.sql**:

![](imagenes/postgis_query_insert_table_comunas.png){width="100%"}




## Base de Datos: PostGIS {.medium-par}

- La BBDD se puebla a partir de scripts **.sql** creados con la herramienta **shp2pgsql**.
  - como parte de un flujo que procesa los datos relevantes desde diferentes fuentes (planificación censal, geometrías comunales, cambios debido a la contingencia, etc)


## Base de Datos: PostGIS {.medium-par}

- La BBDD se puebla a partir de scripts **.sql** creados con la herramienta **shp2pgsql**.
  - como parte de un flujo que procesa los datos relevantes desde diferentes fuentes (planificación censal, geometrías comunales, cambios debido a la contingencia, etc)

TODO: figure with diagram of processing workflow


## Base de Datos: PostGIS {.medium-par}

::: incremental
- Tabla 1: **comunas**
  - Contiene 354 filas con información general de cada comuna del país.
  - Coordenadas de centroides de cada sector comunal (urbana, rural, aldea).
  - Niveles de zoom precalculados en base al área de cada sector, usados para centrar el mapa mostrado en la aplicación.
  - Columna **geometry** contiene perímetros de comunas (en formato binarias), usados para dibujar el contorno comunal mostrado en la aplicación.
:::

![](imagenes/postgis_table_comunas.png){width="100%"}



## Base de Datos: PostGIS {.medium-par}

::: incremental
- Tabla 2: **perimeters_arc**
  - Contiene X filas, con múltiples filas por cada comuna. 
  - Cada fila contiene una geometría que incluye los multi-polígonos obtenidos agrupando por ARC y ciclo censal.
  - Esta es nuestra **Optimización I**, que permite hacer más ligeras las consultas a la BBDD y reducir la cantidad de geometrías que la aplicación debe plotear. 
    - Todo esto alineado a la planificación del censo, que es a nivel de ciclo.
:::

![](imagenes/postgis_table_perimeters_arc.png){width="100%"}



## Base de Datos: PostGIS {.medium-par}

::: incremental
- Tabla 3: **perimeters_arc_ciclo**
  - Contiene X filas, una fila por ciclo y por comuna, es decir, cuatro filas por cada comuna (*idealmente*). 
  - Cada fila contiene una geometría que incluye los multi-polígonos obtenidos agrupando todos los ARCs de una comuna para cada ciclo censal.
  - Esta es nuestra **Optimización II**, que se aplica a comunas con un gran número de ARCs (>300). Esto permite reducir aún más la latencia de la aplicación al consultar la BBDD y plotear las geometrías. 
    - Nació como medida de contención ante la alta demanda de la aplicación.
:::

![](imagenes/postgis_table_perimeters_arc_ciclo.png){width="100%"}



## Base de Datos: PostGIS {.medium-par}

- **Optimización I**: geometrías que contienen multi-polígonos obtenidos agrupando por ARC y ciclo censal.


## Base de Datos: PostGIS {.medium-par}

- **Optimización I**: geometrías que contienen multi-polígonos obtenidos agrupando por ARC y ciclo censal.

TODO: image with an example of original and simplified shapes.



## Base de Datos: PostGIS {.medium-par}

- **Optimización II**: geometrías que contienen multi-polígonos obtenidos agrupando todos los ARCs de una comuna en cada ciclo censal.



## Base de Datos: PostGIS {.medium-par}

- **Optimización II**: geometrías que contienen multi-polígonos obtenidos agrupando todos los ARCs de una comuna en cada ciclo censal.

TODO: image with an example of original and simplified shapes.



## Flask + Frontend {.medium-par}

![](imagenes/app_code.png){width=60% .float-right}

::: incremental
- Como vimos anteriormente, Flask es un _microframework_ para desarrollo web de backend, 
famoso por ser sencillo, liviano y flexible. Es la base de otros frameworks más complejos como FastAPI.

- Flask funciona como backend de la aplicación, comunicándose con el frontend y manejando
las lógicas de la aplicación: ploteo, renderizado programático de valores específicos, formulario.

- Además, como herramienta de visualización se seleccionó Plotly, al ser Open Source. 
  - Anteriormente, se hicieron pruebas con Google, pero fueron desestimadas.

:::

## Flask + Frontend {.medium-par}

::: {.r-stack fragment-index=2}

![](imagenes/frontend_app1.png){.fragment fragment-index=4 .fade-in-then-out .float-right}

![](imagenes/frontend_app2.png){.fragment .fade-in-then-out .float-right}

![](imagenes/frontend_app3.png){.fragment .float-right}
:::

::: {.fragment fragment-index=1 }
- Esta aplicación requirió una comunicación fluida con el frontend. En particular, entre Python
y Javascript, lo que se realizó a través de fetch requests, una herramienta relativamente
reciente nativa de Javascript.
  
  - Esto permite que, por ejemplo, Python genere los datos del gráfico, pero Javascript se encargue del
  ploteo, poniendo la carga sobre los usuarios y no sobre nuestros servidores.

:::  

::: {.fragment fragment-index=2 }  
- Javascript se encarga también de las otras lógicas de la aplicación: funcionamiento de botones,
autocompletado de texto, tooltips, botón "loading", etc.

:::

## Gunicorn {.medium-par}


``` yaml
services:
    flask-censo-1:
      container_name: flask-censo-1
      image: flask-censo
      build: ./flask_app
      command: gunicorn --workers 20 --bind 0.0.0.0:5000 app:app 

```

::: incremental
- Uno de los desafíos era escalar la aplicación y uno de los elementos que ayuda
con esto es Green Unicorn.


- Es un servidor HTTP Web Server Gateway Interface (WSGI).

- Crea tantos procesos trabajadores como cores hayan disponibles en la CPU, que
manejan las requests al servidor en paralelo.

- De esta forma gunicorn es quien sirve la aplicación.

![](imagenes/gunicorn-logo.png){ width=40% .float-right}

- Es ampliamente usado en Python por su simpleza, configurabilidad y eficiencia.

:::

## Nginx {.medium-par}

![](imagenes/nginx.png)

- Nginx actúa como parte de nuestra solución para la escalabilidad y la seguridad de la información.

- Actúa como proxy reverso y web server:
  - Acepta _requests_ de clientes y las distribuye al servidor backend (gunicorn), pasándole
  medidas de seguridad agregadas en su archivo de configuración. Luego, entrega la respuesta
  desde el servidor backend al cliente, haciendo parecer que el servidor proxy mismo entregó
  la respuesta.
  - Maneja el protocolo HTTP, además de servir los archivos estáticos de la página web.
  - El balance de la carga es externalizado a un servicio centralizado del INE.
  
## Nginx | Seguridad de la información {.medium-par}

::: {.fragment fragment-index=1} 
- Como parte de una aplicación de uso masivo, se nos solicitó que estuviera conforme
con medidas de seguridad acorde al estándar de seguridad de OWASP.
:::


::: {.fragment fragment-index=2} 
- Para esto realizamos _penetration testing_ y fuimos solucionando los problemas que
fueron presentándose. Entre ellos:
:::

::::: columns

:::: {.column width="40%" .small-par}

::: {.fragment fragment-index=3} 
- Agregar una _Content Security Policy_, para evitar ataques Cross-Site Scripting (XSS) y Clickjacking.
  
:::
  
::: {.fragment fragment-index=5} 
  - Agregar una _Strict Transport Security_, para forzar el uso de protoclo HTTPS
  
:::

::: {.fragment fragment-index=6} 
  - Eliminar header de servidor, para dificultar atacar vulnerabilidades posibles de nginx
  
:::

::: {.fragment fragment-index=7} 
  - Eliminar todo el CSS y Javascript inline posible, para evitar inyecciones de scripts y estilos.
  
:::

::: {.fragment fragment-index=8} 
  - Agregar cookies Secure, HttpOnly y Samesite con valor "Lax"
  
:::

::: {.fragment fragment-index=9} 
  - Etc
  
:::

::::

:::: {.column width="50%"}

::: r-stack


![](imagenes/csp.png){.fragment fragment-index=3 .fade-in-then-out}

![](imagenes/sts.png){.fragment fragment-index=5 .fade-in-then-out}
![](imagenes/server_header.png){.fragment fragment-index=6 .fade-in-then-out}

![](imagenes/cookies.png){.fragment fragment-index=8} 

:::



::::



:::::


::: notes

- Cookie httponly: no se puede acceder a cookie desde javascript
- cookie secure: cookie solo será enviada desde requests https
- Cookie samesimte: Evita ataques CSRF (Cross-site request forgery)

:::

## Orquestado de contenedores {.medium-par}

![](imagenes/docker-compose.png){.float-right width=45%}

- El orquestado de los contenedores requiere levantar 3 los servicios mencionados en el diagrama
- Originalmente, fue utilizada una sola instancia de flask-gunicorn, sin embargo, esto resultó no ser
del todo eficiente, por lo que finalmente se levantan 3 instancias, que reciben las requests en la medida
que van llegando desde `nginx` a través de un sistema de reparto _round robin_.
- Fuera de esto, la implementación es relativamente sencilla: se montan volúmenes, se utilizan variablees
de ambiente y se aprovechan imágenes pre-existentes.



## Conclusiones

::: incremental

- Gracias al aprovechamiento de las tecnologías Open Source disponibles, fue posible
crear una aplicación de forma colaboorativa, que pudiera adaptarse a los desafíos
que esta presentaba:
  - Postgis aportó en la eficiencia y adaptabilidad de la aplicación.
  - Flask y Python permiten contar con un código fácil de mantener y adaptar.
  - Gunicorn permite escalabilidad.
  - Nginx aporta con medidas de seguridad y escalabilidad

- Todo esto permite que se levante una aplicación acorde a los objetivos y que
ha sido de gran utilidad a la ciudadanía.

:::

## Conversemos

- Queríamos aprovechar de preguntarles qué soluciones han utilizado ustedes ante problemas similares.


#


[<img src="imagenes/logo_portada2.png" width="20%"/>]{.center}


[**Ciencia de datos para el sector público:**]{.big-par .center-justified}
[**Aplicación "Dónde está el censo" y su stack tecnológico**]{.big-par .center-justified}
[**Proyecto Ciencia de Datos**]{.medium-par.center-justified}

[**Abril 2024**]{.big-par .center-justified}



[]{.linea-superior} 
[]{.linea-inferior} 
